In [8]:
import pandas as pd

# Path to CSV file
csv_file_path = 'Salmonella.csv'  
sorted_csv_file_path = 'Salmonella.csv'  # Path for the sorted CSV file

# Read the CSV file
df = pd.read_csv(csv_file_path)

# Checking for duplicates based on 'Plant_Species'
duplicate_count = df.duplicated(subset='Plant_Species').sum()
print(f"Number of duplicate entries based on 'Plant_Species': {duplicate_count}")

# Removing duplicates
df = df.drop_duplicates(subset='Plant_Species')

# Sorting the DataFrame
sorted_df = df.sort_values(by='Plant_Species', ascending=True)

# Save the sorted DataFrame to a new CSV file
sorted_df.to_csv(sorted_csv_file_path, index=False)
print(f"CSV file has been sorted and saved to {sorted_csv_file_path}")

# Count unique plants
unique_plants_count = sorted_df['Plant_Species'].nunique()
print(f"Number of unique plants: {unique_plants_count}")

# Replace 'Salmonella Typhimurium' with the actual column name in your CSV if different
active_column = 'Salmonella Typhimurium'  # Column indicating activity against Salmonella Typhimurium

# Count the number of plants active and not active against Salmonella Typhimurium
active_plants_count = sorted_df[sorted_df[active_column] == 1]['Plant_Species'].nunique()
non_active_plants_count = sorted_df[sorted_df[active_column] == 0]['Plant_Species'].nunique()
print(f"Number of plants active against Salmonella Typhimurium: {active_plants_count}")
print(f"Number of plants not active against Salmonella Typhimurium: {non_active_plants_count}")

# Calculate the percentage of active and non-active plants
percent_active = (active_plants_count / unique_plants_count) * 100
percent_non_active = (non_active_plants_count / unique_plants_count) * 100

print(f"Percentage of plants active against Salmonella Typhimurium: {percent_active:.2f}%")
print(f"Percentage of plants not active against Salmonella Typhimurium: {percent_non_active:.2f}%")


Number of duplicate entries based on 'Plant_Species': 0
CSV file has been sorted and saved to Salmonella.csv
Number of unique plants: 177
Number of plants active against Salmonella Typhimurium: 89
Number of plants not active against Salmonella Typhimurium: 88
Percentage of plants active against Salmonella Typhimurium: 50.28%
Percentage of plants not active against Salmonella Typhimurium: 49.72%


In [9]:
import pandas as pd
import unicodedata
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Normalize text function
def normalize_text(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
    return text.lower().strip()

# Function to capitalize the first word of a string
def capitalize_first_word(text):
    parts = text.split()
    if parts:
        parts[0] = parts[0].capitalize()
    return ' '.join(parts)

# Load and process Sorted_Salmonella.csv
sorted_plants_path = 'Salmonella.csv'
# Load only the first column, assuming it's the 'Plant_Species' column
sorted_df = pd.read_csv(sorted_plants_path, usecols=[0])
sorted_df['Plant_Species'] = sorted_df['Plant_Species'].apply(normalize_text).apply(capitalize_first_word)

# Load and process chemicals_data_clean_unique.csv
chemicals_path = '/Users/mariiakokina/Documents/eo_database/initial_data/chemicals_data_clean_unique.csv'  
chemicals_df = pd.read_csv(chemicals_path)
chemicals_df['Plant_Species'] = chemicals_df['Plant_Species'].apply(normalize_text)

# Function to find the best match with a fuzzy match score and track unmatched plants
def best_match(plant, species_list):
    match_data = process.extractOne(plant, species_list, scorer=fuzz.partial_ratio)
    if match_data:
        best_match, score, _ = match_data
        if score >= 90:
            print(f"Match found: {plant} -> {best_match} with score {score}")
            return best_match
        else:
            print(f"No adequate match found for: {plant}")
            return None
    return None

# Initialize data structures for matched and unmatched plants
matched_plants = pd.DataFrame()
unmatched_plants = []  # List to keep track of unmatched plants

# Iterate through sorted plants to find matches in chemicals DataFrame
for plant in sorted_df['Plant_Species']:
    match = best_match(plant, chemicals_df['Plant_Species'])
    if match:
        # Find the row in chemicals_df that matches
        matched_plant_data = chemicals_df[chemicals_df['Plant_Species'] == match].copy()
        matched_plant_data.loc[:, 'Plant_Species'] = plant  # Set the plant species name
        matched_plants = pd.concat([matched_plants, matched_plant_data])
    else:
        unmatched_plants.append(plant)  # Add to unmatched plants list

# Save the matched DataFrame to a new CSV file
output_file_path = 'matched_plants_data.csv'
matched_plants.to_csv(output_file_path, index=False, sep=',')

# Print out the counts of plants and the list of unmatched plants
original_plants_count = sorted_df.shape[0]
matched_plants_count = matched_plants.shape[0]
print(f"Number of plants in original file: {original_plants_count}")
print(f"Number of plants in matched file: {matched_plants_count}")
print("Unmatched plant species:", unmatched_plants)


Match found: Abies balsamea -> fir balsam 1 - abies balsamea (l.) mill., fam. pinaceae with score 100
Match found: Abies borisii-regis -> abies borisii-regis needle (greece) - abies borisii-regis mattf., fam. pinaceae with score 100
Match found: Abies nordmannia -> abies nordmannia conifer - abies nordmannia (steven) spach, fam. pinaceae with score 100
Match found: Abies semenovii -> abies semenovii - abies semenovii l., fam. pinaceae with score 100
Match found: Achillea ligustica -> achillea ligustica flower (greece) - achillea ligustica all., fam. asteraceae (compositae) with score 100
Match found: Achillea millefolium -> yarrow (canada) 1 - achillea millefolium l., fam. asteraceae (compositae) with score 100
Match found: Aegle marmelos -> aegle marmelos leaf (india) 1 - aegle marmelos (l.) correa, fam. rutaceae with score 100
Match found: Aeollanthus suaveolens -> aeollanthus suaveolens 1a flowering - aeollanthus suaveolens mart. ex spreng., fam. lamiaceae (labiatae) with score 100


Match found: Eruca sativa -> rocket leaf 1 - eruca sativa l., fam. brassicaceae with score 100
Match found: Eucalyptus camaldulensis -> eucalyptus camaldulensis (algeria) - eucalyptus camaldulensis dehn., fam. myrtaceae with score 100
Match found: Eucalyptus globulus -> eucalyptus globulus (algeria) 1 - eucalyptus globulus labill., fam. myrtaceae with score 100
Match found: Eugenia caryophyllus -> clove bud (india) - eugenia caryophyllus (spreng.) bullock & harrison (e.caryophyllata thunb.) with score 100
Match found: Ferula galbaniflua -> galbanum 1 - ferula galbaniflua boiss. et buhse, fam. apiaceae (umbelliferae) with score 100
Match found: Fissitigma shangtzeense -> fissitigma shangtzeense, flower extract - fissitigma shangtzeense tsiang et p.t. li, fam. annonaceae with score 100
Match found: Foeniculum vulgare -> fennel, bitter (china) 1 - foeniculum vulgare miller, var. vulgare dc, fam. apiaceae (umbelliferae) with score 100
Match found: Gardenia jasminoides -> jasmine rose absol

Match found: Picea engelmanni -> picea engelmanni conifer - picea engelmanni (parry) engelm., fam. pinaceae with score 100
Match found: Pimenta dioica -> pimento (allspice) (cuba) 1 (berry) - pimenta dioica (l.) merr. (p. officinalis lindl.), fam. myrtaceae with score 100
Match found: Pimenta racemosa -> bay, west indian (benin) - pimenta racemosa var. racemosa (p. miller) j.w. moore, fam. myrtaceae with score 100
Match found: Pimpinella saxifraga -> pimpinella saxifraga root - pimpinella saxifraga l., fam. apiaceae (umbelliferae) with score 100
Match found: Pinus halepensis -> pinus halepensis (greece) - pinus halepensis miller, fam. pinaceae with score 100
Match found: Pinus mugo -> pinus mugo (dwarf pine) 1 - pinus mugo turra ssp. pumilio (haenke) zenari (p.montana miller), pinaceae with score 100
Match found: Pinus nigra -> pinus nigra - pinus nigra arnold (austrian pine), fam. pinaceae with score 100
Match found: Pinus pinaster -> pinus pinaster (france) - pinus pinaster ait., fam

In [83]:
import pandas as pd

# Define the path to your CSV file
csv_file_path = 'matched_plants_data.csv'

# Read the CSV file, assuming the first row is headers
df = pd.read_csv(csv_file_path)

# Convert columns to numeric (excluding 'Plant_Species' column, assuming it contains labels)
for column in df.columns[1:]:  # Assuming the first column is 'Plant_Species'
    df[column] = pd.to_numeric(df[column], errors='coerce')  # Convert to numeric, setting non-convertible values to NaN

# Replace values less than 0.1 with 0 in numeric columns only
numeric_columns = df.select_dtypes(include=['number']).columns
df[numeric_columns] = df[numeric_columns].applymap(lambda x: 0 if x < 0.09 else x)  

# Drop columns that contain only 0
columns_dropped = df.columns[(df == 0).all()]
df.drop(columns=columns_dropped, inplace=True)

# Update the list of numeric columns after dropping the all-zero columns
numeric_columns = df.select_dtypes(include=['number']).columns

# Keep track of the original indices
original_indices = set(df.index)

# Drop rows that contain only 0 after filtering for numeric columns
df = df.loc[~(df[numeric_columns] == 0).all(axis=1)]

# Find out which indices were dropped
dropped_indices = original_indices - set(df.index)

# Print the columns that were dropped
print("Columns Dropped:", columns_dropped.tolist())
print("Rows Dropped:", dropped_indices)

# Save the modified DataFrame to a new CSV file
df.to_csv('matched_plants_10%.csv', index=False)


Columns Dropped: ['Bornyl acetate', 'Santene', 'alpha-Muurolene', 'Ylangene', 'Longifolene', 'beta-Selinene', 'gamma-Cadinene', 'beta-Cadinene', 'Calamene', 'Tricyclene', 'alpha-Humulene', 'Cadinene (unknown isomer)', 'Terpinolene', 'beta-Himachalene', 'alpha-Terpinene', 'beta-Bisabolene (tent.)', 'alpha-Fenchol', 'Camphene hydrate', 'Piperitone', 'Bornylene', 'Fenchyl acetate', 'gamma-Bisabolene', 'Sibirene', 'alpha-Bisabolene', 'alpha-Longipinene', 'Longicyclene', 'alpha-Ylangene', 'Benzaldehyde', 'Benzyl alcohol', 'Nerol', 'Piperitol', '(-)-Isopulegol', 'Chavibetol', 'Aromadendrene', 'meta-Cymene', 'p-Cymen-8-ol', 'trans-Fragranyl acetate', 'Fragranol', 'cis-Sabinene hydrate', 'trans-Sabinene hydrate', 'Caryophylladienol', 'Fragranyl 2-methylbutyrate', '2,3-dehydro-1,8-Cineole', 'trans-Pinocarveol', '2-Methylbutyl 2-methylbutyrate', 'Terpinen-4-yl acetate', 'delta-Terpineol', 'Fragranyl isovalerate', 'Isoamyl acetate', 'Isoamyl isovalerate', 'Spathulenol', '12-Methyl-13-tridecanolid

In [10]:
import pandas as pd
import re

# Function to extract the core chemical name (excluding parentheses)
def extract_core_name(text):
    return re.sub(r'\s*\([^)]*\)', '', text).strip().lower()

# Load the initial chemicals data
initial_file_path = 'matched_plants_10%.csv' 
initial_df = pd.read_csv(initial_file_path)

# Load the chemicals fingerprints data
fingerprints_file_path = '/Users/mariiakokina/Documents/eo_database/initial_data/matched_chemicals_fingerprints.csv'  # Update with the actual path
fingerprints_df = pd.read_csv(fingerprints_file_path)

# Normalize chemical names for case-insensitive matching
fingerprints_df['Chemical Name'] = fingerprints_df['Chemical Name'].str.lower()

# Dictionary to hold chemical name to fingerprint mapping
chemical_to_fingerprint = dict(zip(fingerprints_df['Chemical Name'], fingerprints_df['Fingerprints']))

# Track the number of columns dropped and their names
dropped_columns_count = 0
dropped_columns_names = []  # List to store names of dropped columns

# Iterate through the initial DataFrame columns and check for fingerprint matches
for col in initial_df.columns[1:]:  # Skip the first column
    normalized_col = col.lower()
    core_name = extract_core_name(normalized_col)

    if core_name not in chemical_to_fingerprint and normalized_col not in chemical_to_fingerprint:
        dropped_columns_count += 1  # Increment the count of dropped columns
        dropped_columns_names.append(col)  # Add the original column name to the list of dropped columns

# Drop the columns from the original DataFrame
initial_df.drop(columns=dropped_columns_names, inplace=True)

# Save the updated original DataFrame to a new CSV file
updated_initial_file_path = 'updated_matched_plants_10%.csv'  # Update with the actual path
initial_df.to_csv(updated_initial_file_path, index=False)

# Print out the number of columns that were dropped and their names
print(f"Number of columns dropped: {dropped_columns_count}")
for col_name in dropped_columns_names:
    print(f"Dropped column: {col_name}")
print(f"Updated initial DataFrame saved to {updated_initial_file_path}")


Number of columns dropped: 18
Dropped column: Cubenol
Dropped column: Sesquiterpene alcohols (unknown)
Dropped column: Bupleurol
Dropped column: Allyl-2,3,4,5-tetramethoxybenzene
Dropped column: alpha-oxo-Bisabolene
Dropped column: Sesquiterpenes, oxygen-containing-
Dropped column: Methyl thymol
Dropped column: Nonadecene/nonadecane
Dropped column: Alkanes & alkenes
Dropped column: Acids and esters
Dropped column: 2-(1,2-Epoxypropane)-5-methoxyphenyl 2-methylbutyrate
Dropped column: Alkanes
Dropped column: cis,p-1(7),8-Menthadienol-2
Dropped column: S-(3-Methylbut-2-enyl) 3-methylbutanethioate
Dropped column: (E)-Cinnamyl aldehyde
Dropped column: 2-Hydroxy-4-methoxy-acetophenone
Dropped column: Agropinene
Dropped column: 3-Vinyl-1,2-dithia-5-cyclohexene
Updated initial DataFrame saved to updated_matched_plants_10%.csv


In [11]:


import pandas as pd
import re

# Function to extract the core chemical name (excluding parentheses)
def extract_core_name(text):
    return re.sub(r'\s*\([^)]*\)', '', text).strip().lower()

# Load the initial chemicals data with semicolon as delimiter
initial_file_path = 'updated_matched_plants_10%.csv'  # Update with the actual path
initial_df = pd.read_csv(initial_file_path)

# Load the chemicals fingerprints data with comma as delimiter
fingerprints_file_path = '/Users/mariiakokina/Documents/eo_database/initial_data/matched_chemicals_fingerprints.csv'  
fingerprints_df = pd.read_csv(fingerprints_file_path)

# Normalize chemical names for case-insensitive matching
fingerprints_df['Chemical Name'] = fingerprints_df['Chemical Name'].str.lower()

# Dictionary to hold chemical name to fingerprint mapping
chemical_to_fingerprint = dict(zip(fingerprints_df['Chemical Name'], fingerprints_df['Fingerprints']))

# Initialize the fingerprint header with placeholders for all columns
fingerprint_header = ['Plant_Species']  # Explicitly set the first column header

# Track the number of columns dropped (excluding the first one)
dropped_columns_count = 0

# Iterate through the initial DataFrame columns and update the fingerprint header
for col in initial_df.columns[1:]:  # Skip the first column
    normalized_col = col.lower()
    core_name = extract_core_name(normalized_col)

    if core_name in chemical_to_fingerprint or normalized_col in chemical_to_fingerprint:
        fingerprint = chemical_to_fingerprint.get(core_name) or chemical_to_fingerprint.get(normalized_col)
        fingerprint_header.append(fingerprint)  # Update the corresponding position in the header
    else:
        dropped_columns_count += 1
        fingerprint_header.append('Unmatched')  # Indicate unmatched columns

# Filter the DataFrame to only include columns with matched fingerprints
matched_columns = ['Plant_Species']  # Start with the first column
matched_columns.extend([col for i, col in enumerate(initial_df.columns[1:], 1) if fingerprint_header[i] != 'Unmatched'])

# Filter the initial DataFrame to only include matched columns
initial_df = initial_df[matched_columns]

# Insert the fingerprint row above the column headers as the new first row
initial_df.columns = pd.MultiIndex.from_tuples(list(zip(fingerprint_header, initial_df.columns)))

# Save the matched DataFrame to a new CSV file
output_file_path = 'matched_chemicals_with_names_fingerprints.csv'  # Update with the actual path
initial_df.to_csv(output_file_path, index=False)  

# Print out the number of columns that were dropped (excluding the first one)
print(f"Number of columns dropped (excluding the first one): {dropped_columns_count}")
print(f"Matched DataFrame saved to {output_file_path}")




Number of columns dropped (excluding the first one): 0
Matched DataFrame saved to matched_chemicals_with_names_fingerprints.csv


In [12]:
import pandas as pd

# file path for the dataset
file_path = 'matched_chemicals_with_names_fingerprints.csv'

# Load the dataset
data = pd.read_csv(file_path)

# Rename the first column
data.rename(columns={data.columns[0]: 'Chemical_Fingerprint'}, inplace=True)

# Save the dataset back out
data.to_csv(file_path, index=False)

print(f"First column renamed and file overwritten at {file_path}")


First column renamed and file overwritten at matched_chemicals_with_names_fingerprints.csv


In [13]:
import pandas as pd
import re

# Function to extract the core chemical name (excluding parentheses)
def extract_core_name(text):
    return re.sub(r'\s*\([^)]*\)', '', text).strip().lower()

# Load the initial chemicals data with colon as delimiter
initial_file_path = 'updated_matched_plants_10%.csv'  # Update with the actual path
initial_df = pd.read_csv(initial_file_path, sep=',')

# Load the chemicals fingerprints data
fingerprints_file_path = '/Users/mariiakokina/Documents/eo_database/initial_data/matched_chemicals_fingerprints.csv'  # Update with the actual path
fingerprints_df = pd.read_csv(fingerprints_file_path)

# Normalize chemical names for case-insensitive matching
fingerprints_df['Chemical Name'] = fingerprints_df['Chemical Name'].str.lower()

# Dictionary to hold chemical name to fingerprint mapping
chemical_to_fingerprint = dict(zip(fingerprints_df['Chemical Name'], fingerprints_df['Fingerprints']))

# Track the number of columns dropped (excluding the first one)
dropped_columns_count = 0

# Iterate through the initial DataFrame columns and replace concentrations with fingerprints
for col in initial_df.columns[1:]:  # Skip the first column
    normalized_col = col.lower()
    core_name = extract_core_name(normalized_col)

    if core_name in chemical_to_fingerprint or normalized_col in chemical_to_fingerprint:
        # Get the fingerprint for the chemical
        fingerprint = chemical_to_fingerprint.get(core_name) or chemical_to_fingerprint.get(normalized_col)
        # Replace entire column's concentration values with the fingerprint
        initial_df[col] = fingerprint
    else:
        # Drop the column from the DataFrame if no match found
        initial_df.drop(columns=[col], inplace=True)
        dropped_columns_count += 1

# Save the modified DataFrame to a new CSV file
output_file_path = 'plants_with_fingerprints.csv'  # Update with the actual path
initial_df.to_csv(output_file_path, index=False)

# Print out the number of columns that were dropped (excluding the first one)
print(f"Number of columns dropped (excluding the first one): {dropped_columns_count}")
print(f"Modified DataFrame saved to {output_file_path}")


Number of columns dropped (excluding the first one): 0
Modified DataFrame saved to plants_with_fingerprints.csv


In [14]:
import pandas as pd

# Define the path to your dataset
dataset_path = '/Users/mariiakokina/Documents/eo_database/chemical_composition/plants_with_fingerprints.csv'

# Read the dataset
df = pd.read_csv(dataset_path)

# Loop through columns (starting from the second column)
for column in df.columns[1:]:
    # Append "_fingerprint" to the column name
    df.rename(columns={column: column + '_fingerprint'}, inplace=True)

# Save the modified dataset with updated column names
df.to_csv('/Users/mariiakokina/Documents/eo_database/chemical_composition/plants_with_fingerprints_updated.csv', index=False)


In [19]:
import pandas as pd

# Define the file paths
file1_path = 'plants_with_fingerprints_updated.csv'
file2_path = 'Salmonella.csv'
file3_path = 'updated_matched_plants_10%.csv'

# Read the CSV files and count the number of rows and columns
df1 = pd.read_csv(file1_path)
df2 = pd.read_csv(file2_path)
df3 = pd.read_csv(file3_path)

# Get the number of rows and columns in each DataFrame
num_rows_file1, num_cols_file1 = df1.shape
num_rows_file2, num_cols_file2 = df2.shape
num_rows_file3, num_cols_file3 = df3.shape

# Compare the number of rows and columns
if num_rows_file1 == num_rows_file2 == num_rows_file3:
    print("All three files have the same number of rows:", num_rows_file1)
else:
    print("The number of rows in the files is not equal.")
    print("File 1 has {} rows.".format(num_rows_file1))
    print("File 2 has {} rows.".format(num_rows_file2))
    print("File 3 has {} rows.".format(num_rows_file3))

if num_cols_file1 == num_cols_file2 == num_cols_file3:
    print("All three files have the same number of columns:", num_cols_file1)
else:
    print("The number of columns in the files is not equal.")
    print("File 1 has {} columns.".format(num_cols_file1))
    print("File 2 has {} columns.".format(num_cols_file2))
    print("File 3 has {} columns.".format(num_cols_file3))


All three files have the same number of rows: 177
The number of columns in the files is not equal.
File 1 has 131 columns.
File 2 has 3 columns.
File 3 has 131 columns.
